# Collective Synchronization: The Kuramoto Model

Author: Ravi Chepuri

This notebook is the first part of a challenge question for the 2023 Winter Workshop of the [GRAD-MAP program](https://www.umdgradmap.org). The challenge question is intended to be a week-long exploration in nonlinear dynamics for physics students at the intermediate undergraduate level who have limited previous Python and coding experience.

This challenge question is inspired by L Q English 2008 Eur. J. Phys. 29 143.

**Learning goals:**
Create a simulation of the Kuramoto model in Python, and investigate the conditions under which spontaneous synchronization occurs.

**Introduction:**
Collective synchronization is a remarkable effect observed in many diverse areas of our world. Synchronization can appear between living things, such as in the [flocking behavior of birds](https://www.youtube.com/watch?v=0dskCpuxqtI) and fish, the [synchronized flashing of some types of fireflies](https://www.youtube.com/watch?v=0BOjTMkyfIA), and even the synchronized pulsing of the cells of your heart to pump blood. But synchronization doesn’t just occur between living things: it can also occur [between pendulums](https://www.youtube.com/watch?v=T58lGKREubo), in the motion of planetary bodies, and in [chemical reactions](https://www.youtube.com/watch?v=PpyKSRo8Iec).

To understand the many phenomena of collective synchronization, a mathematical model called the Kuramoto model has been studied in the research community. In the Kuramoto model, a collection of individual oscillators is considered, where each oscillator wants to oscillate at its own natural frequency. Each oscillator is often represented as a point on a circle, and oscillations are shown by the point moving around the circle. However, each oscillator is also influenced by the motion of the other oscillators, being pushed and pulled in a specific way as expressed by a set of equations called the Kuramoto equations. Due to these interactions, the Kuramoto model exhibits collective behavior. Although it is a relatively simple model without the diverse nuances of real-life spontaneous synchronization, study of the Kuramoto model has still revealed much insight into spontaneous synchronization that can be generalized to many areas, and research related to the Kuramoto model is ongoing.

***In this challenge question, you will use Python to create a computational simulation of the Kuramoto model.*** The first objective of the challenge question is to observe spontaneous synchronization in the results of your simulation. If you achieve this, you can then begin exploring the conditions under which synchronization occurs in the Kuramoto model, changing for example the coupling strength between the oscillators, or the underlying interaction network. Such investigation may lead you to encounters with topics such as phase transitions, network science, and universality.

**Recommended external resources:**
* [Veritasium: The Surprising Secret of Synchronization](https://youtu.be/t-_VPRCtiUg) (the whole video is great but 6:14-8:03 is especially relevant)
* [Wikipedia: Kuramoto Model](https://en.wikipedia.org/wiki/Kuramoto_model)
* [Synchronization of oscillators: an ideal introduction to phase transitions](http://doi.org/10.1088/0143-0807/29/1/015)


![Kuramoto model in action](https://i.imgur.com/4aeM2Jn.gif)

The Kuramoto model in action, showing spontaneous synchronization.

**How to use this notebook:**
The aim of this notebook is to guide you through the process of creating a numerical simulation of the Kuramoto model. 

Part I. contains background information about the Kuramoto model and a few mathematical and coding topics that may be needed to complete the simulation of the Kuramoto model. This info is intended to be a reference for the challenge question, but you are also encouraged to seek information through your own research via the internet. *Feel free to get started on Part II. before fully reading all of this section, and refer back to this Part I. as needed.*

Part II. is where you will create a numerical simulation of the Kuramoto model in guided steps. You will be tasked with completing or creating pieces of Python code, according to the guiding information. The tasks will start out relatively straightforward at first, but will become progressively more open-ended. As you progress through the notebook, you will visualize results using animations. You will also be asked to write brief reflections about what you observe. The reflections may also ask you to modify certain parts of your code and record what happens.

Parts III. and IV. are contained in separate notebooks and focus on phase transitions and network effects in the Kuramoto model, respectively. These parts still consist of guided tasks, but the tasks are significantly more open-ended.

## I. Background Information

### The Kuramoto model

The Kuramoto model is a mathematical model of interacting oscillators. Although the model is relatively simple, it shows rich behaviors that we will get a taste of in this challenge question.

In the Kuramoto model, we consider a collection of oscillators. In the real world, an oscillator might be a pendulum, heartbeat, orbit of a planet, or anything that shows periodic, repeating behavior. Here, we consider abstract oscillators, each of which can be visualized as a dot moving around the unit circle:

![Single Kuramoto oscillator](https://i.imgur.com/VJXwNJw.gif)

The movement of the dot represents the oscillator moving through the phases of its cycles. Every oscillator is given a natural angular frequency, so each tends tends to move around the circle at that given speed. However, each oscillator is also influenced by each of the others: An oscillator's frequency is pushed or pulled, depending on the difference between its own phase and the phase of the other oscillator.

Let's state this more specifically using mathematical equations. Say we have a collection of $N$ oscillators. Denote the phase of the $i$ th oscillator (its angle on the unit circle) at a given time $t$ by $\theta_i(t)$. Denote the natural angular frequency of the $i$ th oscillator by the constant $\omega_i$. Then the (globally connected) Kuramoto model is governed by the set of $N$ equations

\begin{equation}
    \frac{\mathrm{d} \theta_i}{\mathrm{d} t} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} \sin(\theta_j - \theta_i) \quad (i = 1, \ldots, N)
\end{equation}

where $K$ is a constant known as the coupling constant. When $K$ is increased, the strength of the interaction between the oscillators is increased. The meaning of these equations may become more clear as you work through this notebook.

### Differential equations

The $N$ Kuramoto model equations are examples of coupled differential equations. Here we'll quickly review what a differential equation is, and what it means that they're coupled. Based on your previous experience with differential equations you may need additional review of this topic - there are numerous online resources about the basics of differential equations.

A differential equation is an equation that describes the rate of change of a variable. Since derivatives tell you the rate of change of a variable, differential equations involve derivatives. An example of a simple ordinary differential equation is $\frac{dx}{dt} = x$. If we interpret $x$ as, for example, the population of bacteria on a petri dish at time $t$, then this equation tells us that the rate of change of bacteria on the dish is equal to the amount of bacteria already on the dish.

In the Kuramoto model, there are $N$ variables in the model: $\theta_1, \theta_2, \ldots, \theta_N$. The Kuramoto model equations specify the rate of change $\frac{d\theta_i}{dt}$ for each of the $N$ oscillators. Note that for any given oscillator $i$, $\frac{\mathrm{d}\theta_i}{\mathrm{d}t}$ depends not only on its own state $\theta_i$, but also on the states of the other oscillators $\theta_j \, (j = 1, \ldots, N)$. Thus, the motion of each of the oscillators is affected by, or coupled to, the motion of the other oscillators.

### Euler's method for solving differential equations

If we have a differential equation (or multiple coupled differential equations), how can we solve it? There are a vast array of techniques for solving differential equations that apply in many different scenarios. Here, we will use a commonly used technique called Euler's method to approximate a solution to differential equations. Euler's method is convenient because of its relative simplicity and how readily it can be adapted to computational simulations, and it happens to work relatively well for the Kuramoto equations.

Say we have a single differential equation $\frac{\mathrm{d}x}{\mathrm{d}t} = f(x)$, where $f$ is some function, and we know that when $t=t_0$, $x = x_0$. If we want to estimate the value of $x$ at a time $t_1$ shortly after $t_0$, we can use the following steps: 

1. Calculate $f(x_0)$. This then equals $\frac{\mathrm{d}x}{\mathrm{d}t}$, the rate of change of $x$, at and just after $t_0$.
2. Assume the rate of change of $x$ is nearly constant between $t_0$ and $t_1$. Then the graph of $x$ vs. $t$ looks nearly like a line with slope $f(x_0)$, and using the equation for a line we estimate that $x(t_1) = x(t_0) + f(x_0) * (t_1 - t_0)$.

Now, say we want to estimate the value of $x$ at a time $t_2$ shortly after $t_1$. We can repeat the above steps, but starting from $t_1$ instead of $t_0$. By repeating this process for an arbitrarily long time, we can make an estimate of $x$ for $t$ arbitrarily long after $t_0$.

When using Euler's method in a computational setting, time is often divided into equal-sized increments: We set a "timestep" $\Delta t$, let $t_0 = 0$, and let $t_1 = 1 * \Delta t$, $t_2 = 2 * \Delta t$, $t_3 = 3 * \Delta t$, $\ldots$. In this case of evenly spaced timesteps, Euler's method then says that 
\begin{equation}
    x(t_n) = x(t_{n-1}) + f(x) * \Delta t , \quad n = 1,2,3,\ldots.
\end{equation}
If we know the value of $x(t_0)$, we can then use this equation to estimate the value of $x(t_1)$, then use that to estimate the value of $x(t_2)$, and on and on.

Lastly, lets relate this back to the $N$ coupled Kuramoto differential equations. Focusing on the $i$ th oscillator, which obeys $\frac{\mathrm{d} \theta_i}{\mathrm{d} t} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} \sin(\theta_j - \theta_i)$, we can see that by applying Euler's method,
\begin{equation}
    \theta_i(t_n) = \theta_i(t_{n-1}) + \left(\omega_i + \frac{K}{N} \sum_{j=1}^{N} \sin\left(\theta_j(t_{n-1}) - \theta_i(t_{n-1})\right)\right) \Delta t , \quad n = 1,2,3,\ldots.
\end{equation}
Although the equation may look a bit long, the key insight is that if we know all of the $\theta_i$ s at timestep $t_{n-1}$, then we can figure out an approximation for all of the $\theta_i$ s at timestep $t_n$. This method is how we will create our simulations of the Kuramoto model, although we will build up to the full simulation in steps.



### Numpy arrays and vectorization

When doing computational work in Python, it's essential to use the popular library Numpy, which provides efficient operations on arrays of data. In Numpy, an array is a grid of values that are all the same type (e.g. integers, floating point numbers, etc.). Arrays are used to store and manipulate large sets of data in a consistent and efficient manner. In our case, we are will use arrays to store data about the phases of Kuramoto oscillators at many points $t_0, t_1, t_2, \ldots$ over time.

One of the key features of Numpy arrays is their support for vectorization, which is the ability to apply a single operation to multiple elements of an array simultaneously. This is in contrast to traditional loops, where a separate operation must be performed on each element of the array individually. Vectorization can greatly improve the performance of numerical computations, especially when working with large arrays. We will use vectorization as part of our simulations.

If you're not already familiar with Numpy, you are encouraged to look up tutorials and documentation for Numpy — there are many online resources.

### Custom Kuramoto plotting function

In order to visualize what's going on in the Kuramoto model, we will use a custom-made function that creates an animation of Kuramoto oscillators. That function, `animate_kuramoto`, is only responsible for creating the animations *after* you feed it the results of your simulations. `animate_kuramoto` is defined in the separate file `kuramoto_plot.py`.

## II. Kuramoto model simulation

Before we start, we need to import the `numpy` library so that we can use Numpy arrays and associated functions. We also import Pyplot from the Matplotlib library, which is another staple of Python that is useful for graphing. We use the common abbreviations `np` and `plt` as a convention. Finally, we import the custom `animate_kuramoto` function that will allow us to create an animation of Kuramoto oscillators.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from kuramoto_plot import animate_kuramoto

(If you are unable to import `numpy` or `matplotlib` in the cell above, check that you have these two Python packages installed correctly. If you can't import `animate_kuramoto`, make sure you have the file `kuramoto_plot.py` in the same folder as this notebook.)

### Simulation of 1 oscillator

To tackle the simulation of the Kuramoto model, we'll first start by considering only one oscillator. In the case $N = 1$, there is only one differential equation:
\begin{equation*}
    \frac{\mathrm{d}\theta_1}{\mathrm{d}t} = \omega_1.
\end{equation*}
(Check for yourself that when you set $N=1$ in the Kuramoto equations, this is what you get.)

Let's use Euler's method to approximate a solution in the case where the natural angular frequency is $\omega_1 = 1 \mathrm{Hz}$. We'll use a timestep $\Delta t = 0.1 \mathrm{s}$ and a simulation length of $1000$ time steps. 

To begin, we'll first create an empty Numpy array `theta_1` of length $1000$ to store the values of $\theta_1$ at each timestep. We'll then choose a random value between $0$ and $2\pi$, and initialize $\theta_1$ at $t_0 = 0$ to that value (the random value is between $0$ and $2\pi$ because $\theta_1$ is an angle in radians). Note that the value of $\theta_1(t_0)$ can be accessed with the code `theta_1[0]`.

**TASK**: Complete the code below to set $\theta_1(t_0)$ to be equal to a random number between $0$ and $2\pi$. The random number has already been stored in the variable `random_initial_value`, so all you have to do is assign the value of `theta_1[0]` to that number.

In [ ]:
# Start by seting the parameters of the simulation
dt = 0.1  # length of one time step
number_of_time_steps = 1000  # total number of time steps in the simulation

omega_1 = 1

theta_1 = np.zeros(number_of_time_steps)  # Empty array to store values of theta_1
random_initial_value = 2 * np.pi * np.random.rand()  # random number between 0 and 2pi

### YOUR CODE BELOW (replace `None` with appropriate code) ###
theta_1[0] = None

Next, we'll use Euler's method to fill in the rest of the values in the array. In this case, Euler's method says that $\theta_1(t_n) = \theta_1(t_{n-1}) + \omega_1 * \Delta t$ for $n = 1,2,3,\ldots.$

**TASK**: Complete the code below to approximate the solution to the differential equation using Euler's method. You will have to use a `for` loop as provided. The value of $\theta_1(t_n)$ can be accessed with the code `theta_1[n]`, the value of $\theta_1(t_{n-1})$ can be accessed with the code `theta_1[n-1]` the value of $\omega_1$ is stored in the variable `omega_1`, and the timestep $\Delta t$ is stored in the variable `dt`.

In [ ]:
# Start by seting the parameters of the simulation
dt = 0.1  # length of one time step
number_of_time_steps = 1000  # total number of time steps in the simulation

omega_1 = 1

theta_1 = np.zeros(number_of_time_steps)  # Empty array to store values of theta_1
random_initial_value = 2 * np.pi * np.random.rand()  # random number between 0 and 2pi
theta_1[0] = None  ### REPLACE with your code from the previous task ###

for n in range(1, number_of_time_steps):
    ### YOUR CODE BELOW (replace `None` with appropriate code) ###
    theta_1[n] = None

Now that have values for $\theta_1(t)$ stored in the array `theta_1`, let's create an animation to see what's going on. We use the function `animate_kuramoto`, which is custom-made for this challenge question.

(Note that `animate_kuramoto` may take a few seconds or longer to run, depending on your computer. This is because Python's `matplotlib` is relatively slow at drawing shapes. The delay is not from your computer running the simulation, but rather from the computer animating the simulation so that you can see what's going on.)

In [ ]:
animate_kuramoto(theta_1)  # May take a few seconds or longer

**Reflection:** Describe the motion of the single oscillator. What is the effect if the value of $\omega_1$ (`omega_1` in the code) is changed to a different value?

*Insert your reflection here*

### Simulation of 2 non-interacting oscillators

Now let's simulate two Kuramoto oscillators. To start off, we'll consider the case when there's no interaction between them ($K = 0$). Then the Kuramoto equations read
\begin{align*}
    \frac{\mathrm{d}\theta_1}{\mathrm{d}t} &= \omega_1 \\
    \frac{\mathrm{d}\theta_2}{\mathrm{d}t} &= \omega_2.
\end{align*}
(Again, check for yourself that when you set $N=2$ and $K=0$ in the Kuramoto equations, this is what you get.)
Notice that the equations are uncoupled, so each equation can be solved separately.

**Task:** Referencing the code from the one oscillator case, use Euler's method to simulate the two noninteracting oscillators in the case where $\omega_1 = 1 \mathrm{Hz}$ and $\omega_2 = 1.1 \mathrm{Hz}$, and store the results for $\theta_1$ and $\theta_2$ in Numpy arrays `theta_1` and `theta_2`. Use the same simluation parameters ($\Delta t$, number of time steps) as the single oscillator simulation. Because they are noninteracting, Euler's method can be applied to each oscillator separately. (Feel free to copy/paste and modify from previous cells.)

In [ ]:
### YOUR CODE HERE ###

Let's create an animation and see what's going on. If we feed the natural frequencies $\omega_i$ to the `plot_kuramoto` function as a list, it will colorcode the oscillators by their natural frequencies, red being faster and blue being slower.

In [ ]:
animate_kuramoto(theta_1, theta_2, natural_frequencies=[omega_1, omega_2])

**Reflection:** Describe the motion of the two oscillators. Does is make sense to call their motion "uncoupled?" What is the effect if the value of $\omega_1$ and $\omega_2$ are changed?

*Insert your reflection here*

### Simulation of 2 oscillators

OK, now let's turn the interaction between the two oscillators on. When $N = 2$ and $K \neq 0$, the Kuramoto equations are
\begin{align*}
    \frac{\mathrm{d}\theta_1}{\mathrm{d}t} &= \omega_1 + \frac{K}{2} \sin{\left( \theta_2 - \theta_1 \right)}\\
    \frac{\mathrm{d}\theta_2}{\mathrm{d}t} &= \omega_2 + \frac{K}{2} \sin{\left( \theta_1 - \theta_2 \right)}.
\end{align*}
(Again, check for yourself that when you set $N=2$ in the Kuramoto equations, this is what you get.)
The equations are now coupled (ex. $\frac{\mathrm{d}\theta_1}{\mathrm{d}t}$ depends on both $\theta_1$ and $\theta_2$) so they cannot be approached separately. However, we can still use Euler's equation to approximate a solution. Euler's method now says that 
\begin{align*}
    \theta_1(t_n) = \theta_1(t_{n-1}) + \left(\omega_1 + \frac{K}{2} \sin(\theta_2 - \theta_1)\right) * \Delta t, \quad n = 1,2,3,\ldots, \\
    \theta_2(t_n) = \theta_2(t_{n-1}) + \left(\omega_2 + \frac{K}{2} \sin(\theta_1 - \theta_2)\right) * \Delta t, \quad n = 1,2,3,\ldots.
\end{align*}

**TASK**: Complete the code below using Euler's method to approximate the solution to the differential equation in the case where $\omega_1 = 1 \mathrm{Hz}$, $\omega_2 = 1.1 \mathrm{Hz}$, and the coupling constant is $K = 0.5 \mathrm{Hz}$. The same simulation parameters as before are used. The code for $\theta_1$ is given but you should complete the code for $\theta_2$.

In [ ]:
dt = 0.1
number_of_time_steps = 1000

K = 0.5
omega_1 = 1
omega_2 = 1.1

theta_1 = np.zeros(number_of_time_steps)
theta_2 = np.zeros(number_of_time_steps)
theta_1[0] = 2 * np.pi * np.random.rand()
theta_2[0] = 2 * np.pi * np.random.rand()

for n in range(1, number_of_time_steps):
    theta_1[n] = theta_1[n-1] + (omega_1 + K/2 * np.sin(theta_2[n-1] - theta_1[n-1])) * dt
    ### YOUR CODE BELOW ###
    theta_2[n] = None

Again, lets look at an animation.

In [ ]:
animate_kuramoto(theta_1, theta_2, natural_frequencies=[omega_1, omega_2])

**Reflection:** Describe the motion of the two oscillators. Do you see anything different from the noninteracting case? What is the effect if you adjust `K` in the code? What is the effect if you adjust `omega_1` and `omega_2` in the code? (**Note:** If the value of `K` is set too high (much higher than $0.5$), you might see the simulation start to show strange behavior. This is a limitation of our simulation, stemming from the fact that Euler's method provides approximations, rather than true solutions, to the Kuramoto equations. For the purposes of this challenge question, we will try to stick to cases where Euler's method provides a reasonable approximation, so we will avoid setting $K$ too large.)


*Insert your reflection here*

### Simulation of 3 oscillators

In the $N = 3$ case of the Kuramoto model, the Kuramoto equations read
\begin{align*}
    \frac{\mathrm{d}\theta_1}{\mathrm{d}t} &= \omega_1 + \frac{K}{3} \big( \sin{\left( \theta_2 - \theta_1 \right)} + \sin{\left( \theta_3 - \theta_1 \right)} \big)\\
    \frac{\mathrm{d}\theta_2}{\mathrm{d}t} &= \omega_2 + \frac{K}{3} \big( \sin{\left( \theta_1 - \theta_2 \right)} + \sin{\left( \theta_3 - \theta_2 \right)} \big)\\
    \frac{\mathrm{d}\theta_2}{\mathrm{d}t} &= \omega_3 + \frac{K}{3} \big( \sin{\left( \theta_1 - \theta_3 \right)} + \sin{\left( \theta_2 - \theta_3 \right)} \big).
\end{align*}

**Task:** Referencing the code from the two-oscillator case, use Euler's method to simulate the three oscillators in the case where $\omega_1 = 0.9 \mathrm{Hz}$, $\omega_2 = 1.0 \mathrm{Hz}$, $\omega_3 = 1.1 \mathrm{Hz}$, and $K = 0.5 \mathrm{Hz}$. Store the results for $\theta_1$, $\theta_2$, and $\theta_3$ in Numpy arrays `theta_1`, `theta_2`, and `theta_3`. Use the same simluation parameters ($\Delta t$, number of time steps) as the previous simulations.


In [ ]:
### YOUR CODE HERE ###

Again, lets look at an animation.

In [ ]:
animate_kuramoto(theta_1, theta_2, theta_3, natural_frequencies=[omega_1, omega_2, omega_3])

**Reflection:** Describe the motion of the three oscillators. Try adjusting the values of the $\omega_i$, and of $K$ (being careful to not make $K$ too large). What do you observe?

*Insert your reflection here*

### Simulation of 4 oscillators

For $N = 4$ oscillators, what would the Kuramoto equations be?

**Task:** Referencing the code from the two and three oscillator case, use Euler's method to simulate the four oscillators in the case where $\omega_1 = 0.9 \mathrm{Hz}$, $\omega_2 = 1.0 \mathrm{Hz}$, $\omega_3 = 1.0 \mathrm{Hz}$, $\omega_4 = 1.1 \mathrm{Hz}$, and $K = 0.5 \mathrm{Hz}$. Store the results for $\theta_1$, $\theta_2$, $\theta_3$, and $\theta_4$ in Numpy arrays `theta_1`, `theta_2`, `theta_3`, and `theta_4`. Use the same simluation parameters ($\Delta t$, number of time steps) as the previous simulations.

In [ ]:
### YOUR CODE HERE ###

Again, lets look at an animation. What do you observe? Again, try adjusting values of $K$ and the $\omega_i$.

In [ ]:
animate_kuramoto(theta_1, theta_2, theta_3, theta_4, natural_frequencies=[omega_1, omega_2, omega_3, omega_4])

**Reflection:** Describe the motion of the four oscillators. Try adjusting the values of the $\omega_i$, and of $K$ (being careful to not make $K$ too large). What do you observe?

*Insert your reflection here*

### Simulation of 4 oscillators using vectorized code

We'd like to get up to simulating large numbers of Kuramoto oscillators. However, as you may have observed, it is becoming a bit tedious to add more and more oscillators. There is another way to write our simulation code so that we don't need to significantly modify it for larger numbers of oscillators.

Instead of storing our values for $\theta_1, \ldots, \theta_N$ each in a separate 1D Numpy array, we will combine all of the values for all of the $\theta_i$ into one 2D Numpy array called `thetas`. The $i$ th row of the 2D array will correspond to the values of $\theta_{i+1}$. (The $+1$ is due to the fact that Numpy, and most programming languages, starting counting at $0$, whereas we have started at $1$ when labeling the $\theta_i$). As such, the 2D array needs to have a height of $N$ entries and a width of the number of time steps of the simulation.

The value at the $i$ th row and the $n$ th column of `thetas` can be accessed with `thetas[i, n]`. Thus, the value of `thetas[i, n]` should be equal to $\theta_{i+1}(t_n)$.

Also, instead of storing the values of the $\omega_i$ separately, we will store them in a single 1D Numpy array `omegas`.

Using Euler's method, we can once again approximate a solution to the Kuramoto equations for four oscillators in the case where $\omega_1 = 0.9 \mathrm{Hz}$, $\omega_2 = 1.0 \mathrm{Hz}$, $\omega_3 = 1.0 \mathrm{Hz}$, $\omega_4 = 1.1 \mathrm{Hz}$, and $K = 0.5 \mathrm{Hz}$. We can also utilize Python's `sum` function to calculate the sum in the Kuramoto equations, instead of manually adding up terms.

The code below is completed for you, but you should carefully read it to understand how it works.

In [ ]:
# Set simulation parameters
dt = 0.1
number_of_time_steps = 1000

# Set parameters of the Kuramoto equations
N = 4
K = 0.5
omegas = np.array([0.9, 1.0, 1.0, 1.1])  # Natural frequencies are stored in a single, 1D array

# Create a 2D array to hold the values of thetas
# Each row represents an oscillator
# Each column represents a time step
thetas = np.zeros((N, number_of_time_steps))

# Set the 0th column (timestep 0) to random values
thetas[:, 0] = 2 * np.pi * np.random.rand(N)

# Loop over all timesteps, and over all oscillators, and apply Euler's method
for n in range(1, number_of_time_steps):
    for i in range(N):
        thetas[i, n] = thetas[i, n-1] + (omegas[i] + K/N * sum(np.sin(thetas[j, n-1] - thetas[i, n-1]) for j in range(N))) * dt

Lets look at an animation again. Although we used a different method for the simulation, the results should look the same as the previous $N=4$ simulation.

In [ ]:
animate_kuramoto(*thetas, natural_frequencies=omegas)
# The asterix (*) is used to "unpack" the 2D Numpy array `thetas`, and feed each row of `thetas` into the function.
# This is needed since `animate_kuramoto` expects each argument to be a 1D array.

### Simulation of $N = 32$ oscillators

The advantage of the vectorized code above is that it can be relatively easily generalized to higher $N$; that is, higher numbers of oscillators.

When we deal with many oscillators, we don't want to have to manually pick the natural frequency $\omega_i$ of each oscilator. Instead, the standard choice for picking the $\omega_i$ is to assign each of them a random number drawn from a normal distribution, also called a Gaussian distribution, or a bell curve. A normal distribution is specified by two parameters: its mean, which describes the average value, and the standard deviation, which describes how much random values typically vary from the mean.

**Task:** Using the vectorized code for $N=4$ above as a template, use Euler's method to simulate the case when there are $N=32$ Kuramoto oscillators, $K = 0.5 \mathrm{Hz}$, and the $\omega_i$ are picked from a normal distribution with mean $1$ and standard deviation $0.1$. In the code provided, the $\omega_i$ have already been assigned, so you don't have to worry about that. The number $32$ isn't significant, you could do 27 or 38 or any positive whole number instead, though if you make $N$ too large your computer may have trouble finishing the simulation. (**Hint:** This should take relatively little effort if you use copy/paste and some light modification!)

In [ ]:
N = 32
mean_omega = 1
standard_deviation_omega = 0.1
omegas = np.random.default_rng().normal(mean_omega, standard_deviation_omega, (N))
### YOUR CODE BELOW ###

Lets look at an animation again. What do you observe? Try adjusting the mean and standard deviation of omega and seeing if you can observe different behaviors. Also try adjusting the value of $K$.

In [ ]:
animate_kuramoto(*thetas, natural_frequencies=omegas)
# Since we are now animating 32 oscillators, this make take a bit longer than previous animations to run

**Reflection:** Describe the collective motion of the thirty-two oscillators. Try adjusting the values of the mean and standard deviation of the $\omega_i$ to see if you can observe different behaviors. You can also try adjusting the value of $K$ (being careful to not make $K$ too large). What do you observe?

*Insert your reflection here*

If you have coded your simulation correctly and used the given parameter values, you should observe that all oscillators begin at random points along the circle, but they eventually synchronize their oscillations. This is an example of collective synchronization! Although the Kuramoto equations only specify pairwise interactions between the oscillators (each of the $\sin(\theta_j - \theta_i)$ terms is a pairwise interaction between oscillator $i$ and $j$), the oscillators all nevertheless collectively synchronize. This is a great example of how individual entities following simple rules can give rise to collective phenomena. 

If you have played around with the parameter values such as the coupling constant $K$ and the standard deviation of the distribution of the $\omega_i$, you may have observed that synchronization does not always occur. An investigation of the conditions under which synchronization does and doesn't occur leads to even deeper physics with broad implications, and this is the topic of the next notebook.

**Optional task:** Define a function that takes `N`, `K`, `mean_omega`, `standard_deviation_omega`, `dt`, and `number_of_time_steps` as arguments, and performs a simulation of the Kuramoto model with those parameters, as you've done previously. The function should return a 2D Numpy array `thetas` containing the simulation results, and also return a 1D Numpy array `omegas` containing the natural frequencies of the oscillators. Create an animation using this function and check that it works as expected.

In [ ]:
def simulate_kuramoto(N, K, mean_omega, standard_deviation_omega, dt, number_of_time_steps):
    """Performs a simulation of the Kuramoto model using Euler's method.

    Args:
        N: Number of Kuramoto oscillators
        K: Coupling constant
        mean_omega: Average value of the normal distribution that the natural
            frequencies are drawn from
        standard_deviation_omega: Width of the normal distribution that the
            natural frequencies are drawn from
        dt: Timestep used for Euler's method in the simulation
        number_of_time_steps: Length of simulation in units of dt

    Returns:
        A tuple of two objects. The first is a 2D Numpy array containing the
        results of the simulation (element [i, n] is the phase of oscillator i+1
        at time t_n). The second is a 1D Numpy array containing a list of the
        natural frequencies of the N oscillators.
    """
    ### YOUR CODE BELOW ###
    thetas = None
    omegas = None
    return thetas, omegas

### Key observations

Use this section to record your main takeaways from this notebook.

*Insert your reflection here*